## Hexa-Tree

This is an interactive canvas for the puzzle described in [Mathologer "Secret of row 10: a new visual key to ancient Pascalian puzzles"](https://www.youtube.com/watch?v=9JN5f7_3YmQ).

#### Below is some code...

In [ ]:
import sys
import math
from ipywidgets import AppLayout
from ipycanvas import MultiCanvas, hold_canvas

s3 = math.sqrt(3)
size = 500
border = 30
csize = size + 2 * border

colors = ['red', 'green', 'yellow']

canvas = MultiCanvas(3, size=(csize, csize))
background_layer = canvas[0]
c = canvas[1]
interaction_layer = canvas[2]

def hexagon(x0, y0, n, color, cv):
    "Draws a hexagon at (x0, y0) position with specified size and color"
    step_x = size / n
    side = step_x / s3
    step_y = 3 * side / 2
    x = step_x * x0 + step_x * y0 / 2
    y = step_y * y0
    cv.fill_style = colors[color]
    cv.begin_path()
    cv.move_to(x + border, csize - border - y)
    cv.line_to(x + step_x / 2 + border, csize - border - (y - side / 2))
    cv.line_to(x + step_x + border, csize - border - y)
    cv.line_to(x + step_x + border, csize - border - (y + side))
    cv.line_to(x + step_x / 2 + border, csize - border - (y + 3 * side / 2))
    cv.line_to(x + border, csize - border - (y + side))
    cv.fill()

def star(n, cv):
    "Draws the star on the tree's top"
    step_x = size / n
    side = step_x / s3
    center_x = size / 2
    center_y = center_x * s3
    cv.fill_style = '#FFC030'
    cv.font =  str(4 * side) + 'px serif'
    cv.text_align = 'center'
    cv.fill_text('★', center_x + border, csize - border - center_y)


def make_color(a, b, rule='combine'):
    """
        Computes the color of a neighbor hexagon.
        Rule 'combine' means either the same color if 'a'='b' or the third color if 'a' is not 'b'.
        Rule 'mod3' is the Pascal's triangle rule
    """
    if rule == 'combine':
        if a == b:
            return a
        else:
            if a > b:
                a, b = b, a
            if a == 0:
                if b == 1:
                    return 2
                else:
                    return 1
            else:
                return 0
    else:
        return (a + b) % 3
    
def draw_bottom_up(a, cv):
    "Calculates and draws the trianlge from bottom to up"
    n = len(a)
    left = []
    right = []
    a0 = []
    for y in range(n):
        a1 = []
        for x in range(n - y):
            if y == 0:
                c = a[x]
            else:
                c = make_color(a0[x], a0[x + 1])
            hexagon(x, y, n, c, cv)
            a1.append(c)
            if x == 0:
                left.append(c)
            if x == n - y - 1:
                right.append(c)
        a0 = a1
    return left, right

def draw_left_right(a, cv):
    "Calculates and draws the trianlge from left to right and down (it's messy but it works)"
    n = len(a)
    right = []
    bottom = []
    a0 = []
    for x in range(n):
        a1 = []
        for y in range(n - x):
            if x == 0:
                c = a[y]
            else:
                c = make_color(a0[y], a0[y + 1])
            hexagon(x, y, n, c, cv)
            a1.append(c)
            if y == 0:
                bottom.append(c)
            if y == n - x - 1:
                right.append(c)
        a0 = a1
    right.reverse()
    return right, bottom

def draw_right_left(a, cv):
    "Calculates and draws the trianlge from right to left and down (it's messy but it works)"
    n = len(a)
    left = []
    bottom = []
    a0 = []
    for x0 in range(n):
        x = n - x0 - 1
        a1 = []
        for y in range(n - x0):
            if x0 == 0:
                c = a[y]
            else:
                c = make_color(a0[y], a0[y + 1])
            hexagon(x - y, y, n, c, cv)
            a1.append(c)
            if y == 0:
                bottom.append(c)
            if y == n - x0 - 1:
                left.append(c)
        a0 = a1
    left.reverse()
    bottom.reverse()
    return left, bottom

#### More code...

The array `A` below defines the colors of the lower row (now it's 28 red and yellow hexagons).
If the row's size is "magic" then the color of the top hexagon color is determined by lower left and right hexagons ONLY. The "magic" sizes are 4, 10, 28, 82, ...
<br>See the abovementioned Youtube video for the explanation.

**Note:** Unfortunately, with large size jupyter breaks with some cryptic messages (thus, size=82 will not work).

In [ ]:
c.clear()

# 4, 10, 28, 82 are the "magic" sizes
A = [0, 2] * 14

B, C = draw_bottom_up(A, c)
star(len(A), c)

# Test updates in other directions:
"""
C, A = draw_left_right(B, c)
B, A = draw_right_left(C, c)
print('A:', A)
print('B:', B)
print('C:', C)
"""

def on_mouse_down(x, y):
    "Changes the color of the clicked hexagon and redraws all the tringle"
    global A
    global B
    global C
    global L
    with hold_canvas(canvas):
        n = len(A)
        step_x = size / n
        side = step_x / s3
        step_y = 3 * side / 2
        x -= border
        y = csize - y
        y -= border
        i = int(x / step_x)
        j = int(y / step_y)
        if n / 2 - 1 <= i <= n / 2 + 1 and j >= n:
            A = [1] * len(A)
            B, C = draw_bottom_up(A, c)
            interaction_layer.clear()
            return
        if i < 0 or i > len(A):
            return
        if j < 1:
            v = A[i]
            v = (v + 1) % 3
            A[i] = v
            B, C = draw_bottom_up(A, c)
        elif i + 1 > n / 2:
            v = C[j]
            v = (v + 1) % 3
            C[j] = v
            B, A = draw_right_left(C, c)
        else:
            v = B[j]
            v = (v + 1) % 3
            B[j] = v
            C, A = draw_left_right(B, c)
            
        interaction_layer.clear()

interaction_layer.on_mouse_down(None)
interaction_layer.on_mouse_down(on_mouse_down)

### And finally, the tree

All the sides are clickable!
<br>Click on the start to reset the state.

**Note**: If the canvas starts blinking or otherwise malfunctioning just rerun it with ">>" toolbar button or "Kernel/Restart & Run All" menu command.

In [ ]:
AppLayout(center=canvas)

### Triangle calculation as a reversible tranformation

Note that each trianlge side determines the color of all the rest hexagons, including other two sides (if the color combination rule is aplied, not a "mod3" rule). Therefore this can be viewed as a reversible transformation of a row consisting of \[012\] digits.

Indeed, you can easily verify that in many cases one side has simpler pattern that the two others (e.g. in terms of "Run Length Encoder", RLE complexity). Hence the idea for a compression algorithm &ndash; get the data, encode it as a 3-base number, run the calculation and then choose which of the resulted "sides" can be compressed more.

However, unlike [BWT](https://en.wikipedia.org/wiki/Burrows%E2%80%93Wheeler_transform) this transform must show much less power (plus BWT is much faster and has application for inexact pattern matching).